In [1]:
!pip install tavily-python




In [2]:
!pip install langchain langchain-community langgraph transformers accelerate tavily-python



In [3]:
import os
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langchain_core.messages import HumanMessage
from langchain_core.prompts import PromptTemplate
from transformers import pipeline

# Set Tavily API Key
os.environ["TAVILY_API_KEY"] = "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"



In [4]:

#  Define the State Model
class ResearchState(BaseModel):
    query: str
    research_data: str = ""  # Ensure default value is a string
    final_answer: str = ""  # Store the drafted response

# Research Agent (Fetches data from Tavily)
def research_agent(state: ResearchState):
    """Fetches research data using Tavily and formats it into a readable string."""
    print("🔍 Research Agent is gathering information...")

    # Search for information
    search_results = tavily_tool.run(state.query)  # Returns a LIST of dicts

    #  Convert the list of results into a readable string
    formatted_data = "\n".join(
        [f"- {item['title']}: {item['url']}" for item in search_results if 'title' in item and 'url' in item]
    )

    print(" Research complete! Passing data to drafting agent...\n")
    return ResearchState(query=state.query, research_data=formatted_data)

#  Answer Drafting Agent (Uses Hugging Face model)
def drafting_agent(state: ResearchState):
    """Generates a final answer based on research data using Hugging Face."""
    print(" Drafting Agent is generating an answer...")

    if not state.research_data.strip():
        return ResearchState(query=state.query, research_data=state.research_data, final_answer="No relevant information found.")

    # Load Hugging Face model (Change to a different model if needed)
    summarization_pipeline = pipeline("text-generation", model="mistralai/Mistral-7B-Instruct", token="your_huggingface_token")

    # Generate an answer
    response = summarization_pipeline(state.research_data[:500], max_length=200)[0]['generated_text']

    print(" Answer drafting complete!\n")
    return ResearchState(query=state.query, research_data=state.research_data, final_answer=response)



NameError: name 'BaseModel' is not defined

In [ ]:
# Build the AI Workflow using LangGraph
workflow = StateGraph(ResearchState)
workflow.add_node("research", research_agent)
workflow.add_node("drafting", drafting_agent)
workflow.set_entry_point("research")
workflow.add_edge("research", "drafting")
graph = workflow.compile()



In [ ]:
state = ResearchState(query="Latest AI advancements in 2025")

#  Run research and drafting agents in sequence
result = graph.invoke(state)

if "final_answer" in result:
    print(" Final Answer:\n", result["final_answer"])
else:
    print("Error: No final answer generated.")





In [ ]:
 #test case 1
 test_query = "Latest AI advancements in 2025"

# Initialize the Tavily search tool
from langchain_community.tools import TavilySearchResults
tavily_tool = TavilySearchResults()

# Run a search query
search_results = tavily_tool.run(test_query)

# Print results
print(" Research Agent Output:\n", search_results)


In [ ]:
#test to see if graph.invoke(state) gives a structures response
state = ResearchState(query="Latest AI advancements in 2025")
result = graph.invoke(state)

# Print the raw result
print("Full System Output:\n", result)

